In [ ]:
# Step 1: Install and Import Libraries
!pip install tensorflow datasets numpy

import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.metrics import categorical_accuracy
import pickle
from datasets import load_dataset

# Step 2: Load the WikiText Dataset
dataset = load_dataset('wikitext', 'wikitext-103-raw-v1')

# Step 3: Limit the dataset size
# You can limit the dataset by taking the first N lines or characters
# Example: Use only the first 1000 lines of text from the dataset
data = ' '.join(dataset['train']['text'][:1000])  # Limit to first 1000 lines

# Optional: Or limit the length of the text (e.g., 5000 characters)
# data = data[:5000]

# Step 4: Preprocess the Data
# Clean the data by removing unwanted characters (e.g., newline, Unicode, etc.)
data = data.replace('\n', ' ').replace('\r', ' ').replace('\ufeff', ' ')

# Step 5: Tokenize the Text
tokenizer = Tokenizer()
tokenizer.fit_on_texts([data])

# Save the tokenizer for later use in predictions
pickle.dump(tokenizer, open('token.pkl', 'wb'))

# Print the size of the vocabulary (number of unique words)
vocab_size = len(tokenizer.word_index) + 1  # +1 to account for padding
print(f"Vocabulary size: {vocab_size}")

# Step 6: Prepare Sequences of Words
sequence_data = tokenizer.texts_to_sequences([data])[0]  # Convert text to sequence of tokens
sequences = []

# Loop through the data to create sequences of 4 tokens (3 for X and 1 for y)
for i in range(3, len(sequence_data)):
    seq = sequence_data[i-3:i+1]  # Create a sequence of 4 words
    sequences.append(seq)

# Convert list of sequences into a NumPy array
sequences = np.array(sequences)

# Split sequences into input (X) and output (y)
X = sequences[:, 0:3]  # First 3 words as input
y = sequences[:, 3]    # 4th word as the output (next word)

# One-hot encode the output labels (y) into a vector format
y = to_categorical(y, num_classes=vocab_size)

# Step 7: Build the LSTM Model
model = Sequential()
model.add(Embedding(vocab_size, 50, input_length=3))  # Use 50 dimensions for the embeddings
model.add(LSTM(100, return_sequences=False))  # 100 units in LSTM
model.add(Dense(100, activation='relu'))
model.add(Dense(vocab_size, activation='softmax'))

# Step 8: Compile the Model
model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.001), metrics=[categorical_accuracy])

# Step 9: Set Up Model Checkpoint (Save Best Model)
checkpoint = ModelCheckpoint("next_word.h5", monitor='loss', verbose=1, save_best_only=True)

# Step 10: Train the Model
# Train the model for 20 epochs with a batch size of 64.
history = model.fit(X, y, epochs=10, batch_size=64, callbacks=[checkpoint])

# Step 11: Evaluate the Model
# Evaluate the model to get the accuracy on the training data
loss, accuracy = model.evaluate(X, y, verbose=1)
print(f"Model Accuracy: {accuracy * 100:.2f}%")

# Step 12: Predict the Next Word
def predict_next_word(model, tokenizer, text):
    # Convert the input text to a sequence of integers using the tokenizer
    sequence = tokenizer.texts_to_sequences([text])
    sequence = np.array(sequence)  # Convert to numpy array

    # Predict the next word (returns the index of the predicted word)
    pred_index = np.argmax(model.predict(sequence), axis=-1)

    # Reverse the word index to get the actual word
    for word, index in tokenizer.word_index.items():
        if index == pred_index:
            return word

# Example: Predict the next word after the input "the cat sat"
text = 'the cat sat'
predicted_word = predict_next_word(model, tokenizer, text)
print(f"The predicted next word is: {predicted_word}")

# Step 13: Interactive Prediction
while True:
    text = input("Enter your line (or type '0' to exit): ")

    if text == "0":
        print("Execution completed....")
        break
    else:
        try:
            text = text.split(" ")
            text = text[-3:]  # Use only the last 3 words
            predicted_word = predict_next_word(model, tokenizer, text)
            print(f"The predicted next word is: {predicted_word}")
        except Exception as e:
            print("Error occurred:", e)
            continue


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.2/69.2 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 44.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.9/241.9 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.6/124.6 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 205.1/205.1 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.7/319.7 kB 15.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
  

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/733k [00:00<?, ?B/s]

train-00000-of-00002.parquet:   0%|          | 0.00/157M [00:00<?, ?B/s]

...

Generating train split:   0%|          | 0/1801350 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3760 [00:00<?, ? examples/s]

Vocabulary size: 7641
Epoch 1/10
721/722 [============================>.] - ETA: 0s - loss: 7.3610 - categorical_accuracy: 0.0754
Epoch 1: loss improved from inf to 7.36040, saving model to next_word.h5
722/722 [==============================] - 34s 44ms/step - loss: 7.3604 - categorical_accuracy: 0.0753
Epoch 2/10
  3/722 [..............................] - ETA: 34s - loss: 6.7815 - categorical_accuracy: 0.0625

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


720/722 [============================>.] - ETA: 0s - loss: 6.9153 - categorical_accuracy: 0.0761
Epoch 2: loss improved from 7.36040 to 6.91552, saving model to next_word.h5
722/722 [==============================] - 31s 43ms/step - loss: 6.9155 - categorical_accuracy: 0.0761
Epoch 3/10
722/722 [==============================] - ETA: 0s - loss: 6.6901 - categorical_accuracy: 0.0911
Epoch 3: loss improved from 6.91552 to 6.69008, saving model to next_word.h5
722/722 [==============================] - 31s 43ms/step - loss: 6.6901 - categorical_accuracy: 0.0911
Epoch 4/10
721/722 [============================>.] - ETA: 0s - loss: 6.3745 - categorical_accuracy: 0.1074
Epoch 4: loss improved from 6.69008 to 6.37472, saving model to next_word.h5
722/722 [==============================] - 31s 43ms/step - loss: 6.3747 - categorical_accuracy: 0.1073
Epoch 5/10
721/722 [============================>.] - ETA: 0s - loss: 6.1111 - categorical_accuracy: 0.1173
Epoch 5: loss improved from 6.37472 to 